# Warsztaty badawcze

## Budowa systemu rekomendacyjnego na podstawie danych z OLX Praca z wykorzystaniem metody ALS

#### Wiktoria Boguszewska, Mateusz Zacharecki, Patrycja Żak

## ALS

Algorytm ALS (Alternating Least Squares) rozkłada macierz ocen na dwie macierze: cech użytkowników i przedmiotów, minimalizując różnicę między rzeczywistymi ocenami a przewidywaniami. ALS działa naprzemiennie – optymalizuje najpierw jedną z tych macierzy (np. cech użytkowników), zakładając, że druga jest stała, a następnie odwrotnie.

Optymalizacja odbywa się za pomocą regresji liniowej z regularyzacją, co poprawia stabilność modelu. Proces ten powtarza się, aż model osiągnie satysfakcjonującą zbieżność.

Zalety:

- Skalowalność i możliwość równoległego przetwarzania.
- Dobrze radzi sobie z rzadkimi danymi (np. brak ocen).

Wady:
- Kosztowny obliczeniowo dla bardzo dużych danych.
- Może utknąć w lokalnym minimum, nie zawsze osiągając optymalny wynik.

# Kod

Biblioteki do pobrania


In [1]:
!pip install implicit

In [2]:
!pip install scikit-optimize

In [3]:
!pip install tqdm

In [4]:
import pandas as pd
import numpy as np
import random
import implicit
from scipy.sparse import coo_matrix
from sklearn.base import BaseEstimator
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


Pracujemy z tym zbiorem danych (szczegóły w linku): https://www.kaggle.com/datasets/olxdatascience/olx-jobs-interactions

In [5]:
# Krok 1: Przetwarzanie wstępne

# Wczytanie pliku
data = pd.read_csv("interactions.csv")
data.head()

# Filtracja
data = data[data['event'] == 'click']

In [6]:
data.dtypes

user          int64
item          int64
event        object
timestamp     int64
dtype: object

In [7]:
data.head()

,user,item,event,timestamp
0,27901,56865,click,1581465600
1,124480,115662,click,1581465600
2,159509,5150,click,1581465600
3,188861,109981,click,1581465600
4,207348,88746,click,1581465600


In [8]:
# Samplowanie po user_id i wybór 10% obiektów o największej liczbie interakcji
user_unique = data['user'].unique()
sample_size = int(len(user_unique) * 0.1)
random_user_10 = random.sample(list(user_unique), sample_size)
df_10 = data[data['user'].isin(random_user_10)]

df_10_dist = df_10[['item','user']].drop_duplicates()
item_count = df_10_dist.groupby('item').size().reset_index(name='count')
item_count = item_count.sort_values(by = 'count', ascending = False)
item_unique = df_10['item'].unique()
sample_size = int(len(item_unique) * 0.1)
top_item_10 = item_count.iloc[:sample_size, :]
df_10_10 = df_10[df_10['item'].isin(top_item_10['item'])]

# Podział zbioru na train i test - tzw. temporal split - zbiorem testowym jest 20% ostatnich interakcji (sortowanie po timestamp) czyli na podstawie 80% wcześniejszych interakcji przewidujemy następne

df_10_10 = df_10_10.sort_values('timestamp')
train_size = int(len(df_10_10) * 0.8)
train_data = df_10_10[:train_size]
test_data = df_10_10[train_size:]

In [9]:
# Tworzymy mapowanie z user i item do indeksów
user_map = {u: i for i, u in enumerate(train_data['user'].unique())}
item_map = {i: j for j, i in enumerate(train_data['item'].unique())}

# Mapujemy ID użytkowników i przedmiotów na indeksy
train_data['user_idx'] = train_data['user'].map(user_map)
train_data['item_idx'] = train_data['item'].map(item_map)

# Tworzymy macierz rzadką użytkownik-przedmiot dla treningu
train_matrix = coo_matrix(
    (np.ones(len(train_data)), (train_data['user_idx'], train_data['item_idx'])),
    shape=(len(user_map), len(item_map))
)

/tmp/ipykernel_5993/3756790464.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['user_idx'] = train_data['user'].map(user_map)
/tmp/ipykernel_5993/3756790464.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['item_idx'] = train_data['item'].map(item_map)


In [10]:
len(train_data)

1108285

In [11]:
# Definiujemy wrapper dla ALS
class ALSWrapper(BaseEstimator):
    def __init__(self, factors=50, regularization=0.01, iterations=15):
        self.factors = factors
        self.regularization = regularization
        self.iterations = iterations
        self.model = None

    def fit(self, X, y=None):
        self.model = implicit.als.AlternatingLeastSquares(factors=self.factors, 
                                                          regularization=self.regularization, 
                                                          iterations=self.iterations)
        self.model.fit(X)
        return self

    def recommend(self, user, user_items, N=10):
        return self.model.recommend(user, user_items, N=N)

    def get_params(self, deep=True):
        return {"factors": self.factors, "regularization": self.regularization, "iterations": self.iterations}

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self

In [12]:
# Definiujemy funkcję do obliczania metryk
def precision_recall_accuracy_f1_at_k(model, train_matrix_csr, test_data, user_map, item_map, k=10):
    correct_recommendations = 0
    total_relevant_items = 0
    total_recommended_items = 0
    total_interactions = 0  # dla accuracy

    total_items = train_matrix_csr.shape[1]  # liczba wszystkich przedmiotów

    for user_id in test_data['user'].unique():
        # Sprawdzenie, czy użytkownik jest w mapie user_map
        if user_id not in user_map:
            continue

        user_idx = user_map[user_id]

        # Upewnij się, że user_idx nie przekracza rozmiaru macierzy
        if user_idx >= train_matrix_csr.shape[0]:
            continue

        # Pobieramy tylko wiersz macierzy CSR dla danego użytkownika
        user_interactions = train_matrix_csr[user_idx]

        # Rekomendacje dla użytkownika
        recommended_items = model.recommend(user_idx, user_interactions, N=k)[0]

        # Rzeczywiste interakcje użytkownika w zbiorze testowym
        relevant_items = test_data[test_data['user'] == user_id]['item'].map(item_map).dropna().values

        if len(relevant_items) == 0:
            continue

        # Liczba trafień w rekomendacjach
        hits = np.isin(recommended_items, relevant_items).sum()

        correct_recommendations += hits
        total_recommended_items += k
        total_relevant_items += len(relevant_items)
        total_interactions += total_items  # Wszystkie możliwe interakcje dla użytkownika

    precision_at_k = correct_recommendations / total_recommended_items
    recall_at_k = correct_recommendations / total_relevant_items
    accuracy_at_k = correct_recommendations / total_interactions

    # Obliczenie F1-score
    if precision_at_k + recall_at_k > 0:
        f1_at_k = 2 * (precision_at_k * recall_at_k) / (precision_at_k + recall_at_k)
    else:
        f1_at_k = 0

    return precision_at_k, recall_at_k, accuracy_at_k, f1_at_k

In [13]:
# Definiujemy klasę scorer
class PrecisionAtKScorer:
    def __init__(self, test_data, user_map, item_map, k=10):
        self.test_data = test_data
        self.user_map = user_map
        self.item_map = item_map
        self.k = k

    def __call__(self, estimator, X, y=None):
        precision, recall, accuracy, f1 = precision_recall_accuracy_f1_at_k(
            estimator.model, X, self.test_data, self.user_map, self.item_map, k=self.k)
        return precision # miara do RandomSearchCV

In [14]:
# Normalizacja wartości współczynników w macierzy
train_matrix = (train_matrix.T * 100).T  # Skaluje wartości dla algorytmu
# Konwersja macierzy COO na CSR
train_matrix_csr = train_matrix.tocsr()

In [ ]:
# Przygotowanie danych treningowych i testowych oraz macierzy
# ... [tworzenie train_matrix_csr, test_data, user_map, item_map] ...

# Definiujemy przestrzeń hiperparametrów
param_als = {
    'factors': (10, 200),  # Zakres liczby czynników
    'regularization': (0.001, 0.1, 'log-uniform'),  # Regularyzacja z log-uniform distribution
    'iterations': (10, 100)  # Zakres liczby iteracji
}

# Tworzymy instancję wrappera ALS
als_wrapper = ALSWrapper()

# Tworzymy instancję naszego scorer'a
precision_scorer = PrecisionAtKScorer(test_data, user_map, item_map, k=10)

# Tworzymy RandomizedSearchCV
opt = RandomizedSearchCV(
    als_wrapper,
    param_als,
    n_iter=10,  # Liczba losowych prób
    scoring=precision_scorer,
    cv=5,  # Cross-validation
    random_state=42  # Aby uzyskać powtarzalność wyników
)

# Trenujemy model i optymalizujemy hiperparametry
opt.fit(train_matrix_csr)

# Wyświetlamy najlepsze wyniki optymalizacji
print(f"Najlepsze parametry: {opt.best_params_}")
print(f"Najlepszy wynik: {opt.best_score_}")

  0%|          | 0/100 [00:00<?, ?it/s]

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 969, in _score
    scores = scorer(estimator, X_test, **score_params)
  File "/tmp/ipykernel_19865/1017961213.py", line 10, in __call__
    precision, recall, accuracy, f1 = precision_recall_accuracy_f1_at_k(
  File "/tmp/ipykernel_19865/3677554078.py", line 8, in precision_recall_accuracy_f1_at_k
    users = np.unique(X_test.nonzero()[0])
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pandas/core/generic.py", line 5902, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'DataFrame' object has no attribute 'nonzero'

  warnings.warn(


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 886, in _fit_and_score
    estimator.fit(X_train, **fit_params)
  File "/tmp/ipykernel_19865/2929555392.py", line 13, in fit
    self.model.fit(X)
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/implicit/cpu/als.py", line 163, in fit
    solver(
  File "_als.pyx", line 148, in implicit.cpu._a

  0%|          | 0/100 [00:00<?, ?it/s]

Najlepsze parametry: {'regularization': 0.1, 'iterations': 100, 'factors': 200}
Najlepszy wynik: nan


In [ ]:
best_model = opt.best_estimator_
precision_at_k, recall_at_k, accuracy_at_k, f1_at_k = precision_recall_accuracy_f1_at_k(
    best_model, train_matrix_csr, test_data, user_map, item_map, k=10
)

# Wszystkie miary dla najlepszego modelu
print(f"Precision: {precision_at_k}")
print(f"Recall: {recall_at_k}")
# print(f"Accuracy: {accuracy_at_k}")
print(f"F1-score: {f1_at_k}")